import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
try:
    mydb.amazon_product_metadata.drop()
finally:
    my_amazon_product_metadata = mydb['amazon_product_metadata']
result = mydb.amazon_product_metadata.create_index([('asin', pymongo.ASCENDING)],unique=True)


In [7]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)



#f = open("output.strict", 'w') 
for l in parse("../../download/meta_Cell_Phones_and_Accessories.json.gz"): 
    #print(l + '\n')
    my_amazon_product_metadata.insert_one(l)

In [8]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']


In [9]:
try:
    mydb.amazon_product_reviews.drop()
finally:
    my_amazon_product_reviews = mydb['amazon_product_reviews']
    result = my_amazon_product_reviews.create_index([('asin', pymongo.ASCENDING)],unique=False)
    my_amazon_product_reviews.create_index([
    ("asin", pymongo.ASCENDING),
    ("reviewerID", pymongo.ASCENDING),
    ("reviewTime", pymongo.ASCENDING)   
])

In [10]:
#{ "title": {"$regex": "^Apple iPhone 5s, "}, "categories" : "Cell Phones" }
#B00F3J4HCA - 0 reviews
#B00F3J4B5S - 342 reviews
#B00F3J4E5U
for l in parse("../../download/word2vec/reviews_Cell_Phones_and_Accessories.json.gz"): 
    #print(l + '\n')
    #TODO: retrieve category from product_mata_data
    product_metadata_cursor = mydb.amazon_product_metadata.find({"asin": l["asin"]})
    first_matching_product_metadata = product_metadata_cursor[0]
    l["categories"] = first_matching_product_metadata["categories"][0]
    if "title" in first_matching_product_metadata.keys():
        l["title"] = first_matching_product_metadata["title"]
        my_amazon_product_reviews.insert_one(l)
